### Parse Swiss data

In [169]:
import difflib
import requests
import pandas
from bs4 import BeautifulSoup

cookies = {
    'JSESSIONID': '4E97C0E435F950E02112A7ADA2171F29.wispo2worker',
}

data = [
  ('formular_set', 'formular_set'),
  ('autoScroll', ''),
  ('hiddenActivetabId', '3'),
  ('sort_by', 'LOCATION_ASC'),
  ('wispo_area_search_string', ''),
  ('wispo_area_search_key', ''),
  ('wispo_area_restore_key', ''),
  ('radius_selector', '10'),
  ('check02', 'true'),
  ('search_string', 'Saisir un mot-cl\xE9'),
  ('javax.faces.ViewState', '6281140167725445556:-1337640028376822279'),
  ('javax.faces.source', 'moreResult'),
  ('javax.faces.partial.event', 'click'),
  ('javax.faces.partial.execute', 'moreResult moreResult'),
  ('javax.faces.partial.render', 'main'),
  ('javax.faces.behavior.event', 'action'),
  ('javax.faces.partial.ajax', 'true'),
]

r = requests.post('https://snow.myswitzerland.com/index.jsf', cookies=cookies, data=data)
data = r.text

'<partial-response id="j_id1"><error><error-name>class javax.faces.application.ViewExpiredException</error-name><error-message><![CDATA[viewId:/index.jsf - Ansicht /index.jsf konnte nicht wiederhergestellt werden.]]></error-message></error></partial-response>'

In [161]:
soup = BeautifulSoup(data, "html5lib")

In [162]:
table = soup.find('table', { "class" : "sportsinfo" })
stations = table.find_all('tr')

AttributeError: 'NoneType' object has no attribute 'find_all'

In [115]:
class Station:
    def __init__(self, name, temp, snow_slopes, snow_station):
        self.name = name
        self.temp = temp
        self.snow_slopes = snow_slopes
        self.snow_station = snow_station
        
    def __str__(self):
        return self.name + " " + self.temp + " " + self.snow_slopes + " " + self.snow_station
    
def extract_station_data(station):
    name = station.find('td', {'class': 'location'}).find('h3').get_text().strip()
    temp = station.find('td', {'class': 'weather info_set1'}).find('em').get_text().strip()
    snow_slopes = station.find('td', {'class': 'info_set2'}).find('em').get_text().strip()
    snow_station = station.find('td', {'class': 'info_set2'}).find('span').get_text().replace('dans la station', '').strip()
    return Station(name, temp, snow_slopes, snow_station)

stations_data = [extract_station_data(station) for station in stations]

### Parse data from MagicPass

In [117]:
MP_URL = "https://www.magicpass.ch/fr/stations/"

mp_raw_data = requests.get(MP_URL).text
soup = BeautifulSoup(mp_raw_data, 'html5lib')

mp_stations = soup.find_all('div', {'class': 'station-item'})
mp_stations = [station.find('span', {'class': 'station-name'}).get_text().strip() for station in mp_stations]

### Match stations

In [159]:
matched = []
def filter_stations(station, names):
    if station.name in names: 
        matched.append(station.name)
        return True
    for name in names:
        if name in station.name:
            matched.append(name)
            return True
    matches = difflib.get_close_matches(station.name, names)
    #print(station.name, matches)
    result = len(matches) == 1
    if result == True:
        matched.append(matches[0])
    return result

def rename_station(station):
    renames = {
        'Les Diablerets - Glacier 3000': "Too Expensive",
        'Jaunpass - Boltigen': 'Much duplicate',
        
        'Les Mosses-La Lécherette': 'La Lécherette',
        'Leysin-Mosses-Lécherette': 'Les Mosses'
    }
    if station.name in renames:
        station.name = renames[station.name]
    return station
        

stations_data = [rename_station(station) for station in stations_data]
matches = list(filter(lambda station: filter_stations(station, mp_stations), stations_data))
unmatched = [x for x in mp_stations if x not in matched]

print("Warning, could not match: ", unmatched)

Warning, could not match:  ['Mayens de Conthey', 'Nax – Mt-Noble', 'Tramelan']
